In [5]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import Ollama
llm_model = Ollama(model = "llama2-uncensored")

/var/folders/mt/c6qz9wtd1813dmvdwcp8l1780000gn/T/ipykernel_1114/620325615.py:4: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm_model = Ollama(model = "llama2-uncensored")


In [6]:
response = llm_model.invoke("Who is rohit sharma and what team does he play for in ipl 2025")
print("Ollama says:", response)

Ollama says: Rohit Sharma is an Indian cricketer who plays as a batsman for the Mumbai Indians in the Indian Premier League (IPL). He is also the captain of the national team, and is regarded as one of the best batsmen in the world.


In [7]:
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [8]:
loader = WebBaseLoader("https://en.wikipedia.org/wiki/Amazon_(company)")

In [9]:
def WebsiteLoader(urls):
    loader = WebBaseLoader(urls)
    return loader.load()

In [10]:
urls = ["https://en.wikipedia.org/wiki/Amazon_(company)", "https://en.wikipedia.org/wiki/Apple_Inc."]
website_docs = WebsiteLoader(urls)


In [11]:
#splitting documents into chunks
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 100,
    length_function = len,
    is_separator_regex = False,
)

all_documents = website_docs

split_documents = text_splitter.split_documents(all_documents)

In [14]:
# Embedding Documents and storing data in chroma

#initiliazing Ollama embeddings
from langchain_ollama import OllamaEmbeddings
embeddings = OllamaEmbeddings(
    model="llama2-uncensored",
)
persist_directory = "./chroma_db"

#store documents in chromadb
vectorstore = Chroma.from_documents(
    documents = split_documents,
    embedding = embeddings,
    persist_directory = persist_directory
)

vectorstore.persist()
print("data succesfully stored in chromadb")

#Reload the vector store for retrieval
vectorstore = Chroma(persist_directory=persist_directory, embedding_function=embeddings)
print("Chromadb reloaded successfully")

In [13]:
#retrieving most relevant documents using similarity search
# from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
retriever = vectorstore.as_retriever(search_type = "similarity", search_kwargs={"k":5})
qa = RetrievalQA.from_chain_type(
    llm = llm, chain_type = "stuff", retriever=retriever, return_source_documents=True)
query = "query?"
result = qa({"query":query})
print(result['result'])

#comments added
#comments2 added
#comment3



NameError: name 'vectorstore' is not defined